In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt


root_dir = '/disk1/labeled/'
npyList = os.listdir(root_dir)
npyList = [root_dir+n for n in npyList]
result = []
label = []
for n,array in enumerate(npyList):
    result.append(np.load(array))
    label.append([n]*result[n].shape[0])
    
result = np.concatenate(result)
label = np.concatenate(label)

In [12]:
X = np.asarray([x.transpose((2,0,1)) for x in result])
X = X.astype(np.float32)/(255.0/2) - 1.0

In [4]:
print(__doc__)

from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

np.random.seed(42)


data = result.reshape(result.shape[0],-1)

n_samples, n_features = data.shape
labels = label
n_digits = len(np.unique(label))

sample_size = 300

print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))


print(79 * '_')
print('% 9s' % 'init'
      '    time  inertia    homo   compl  v-meas     ARI AMI  silhouette')


def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))

bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
              name="k-means++", data=data)

bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
              name="random", data=data)

# in this case the seeding of the centers is deterministic, hence we run the
# kmeans algorithm only once with n_init=1
pca = PCA(n_components=n_digits).fit(data)
bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
              name="PCA-based",
              data=data)
print(79 * '_')

Automatically created module for IPython interactive environment
n_digits: 4, 	 n_samples 1971, 	 n_features 3072
_______________________________________________________________________________
init    time  inertia    homo   compl  v-meas     ARI AMI  silhouette
k-means++   7.39s    10954399027   0.190   0.162   0.175   0.141   0.161    0.119
   random   6.94s    10954565373   0.188   0.160   0.173   0.140   0.159    0.124
PCA-based   1.04s    10956567550   0.186   0.157   0.170   0.141   0.156    0.136
_______________________________________________________________________________


In [8]:
r=result.reshape(result.shape[0],-1)
print len(r)

1971


In [31]:
a = KMeans(init='k-means++', n_clusters=5, n_init=10)
a.fit(data)
prelabel = a.labels_
print prelabel

[3 2 2 ..., 2 4 2]


In [33]:
l0,l1,l2,l3,l4=0,0,0,0,0
for i in labels:
    if i==0:
        l0=l0+1
    if i==1:
        l1=l1+1
    if i==2:
        l2=l2+1
    if i==3:
        l3=l3+1
    if i==4:
        l4=l4+1
print l0,l1,l2,l3,l4

 742 477 34 718 0


In [30]:
l0,l1,l2,l3=0,0,0,0
a,b,c,d=742,477,34,718
for i in prelabel[0:a]:
    if i==0:
        l0=l0+1
    if i==1:
        l1=l1+1
    if i==2:
        l2=l2+1
    if i==3:
        l3=l3+1  
L0=[l0,l1,l2,l3]
l0,l1,l2,l3=0,0,0,0
for i in prelabel[a:a+b]:
    if i==0:
        l0=l0+1
    if i==1:
        l1=l1+1
    if i==2:
        l2=l2+1
    if i==3:
        l3=l3+1  
L1=[l0,l1,l2,l3]
l0,l1,l2,l3=0,0,0,0
for i in prelabel[a+b:a+b+c]:
    if i==0:
        l0=l0+1
    if i==1:
        l1=l1+1
    if i==2:
        l2=l2+1
    if i==3:
        l3=l3+1  
L2=[l0,l1,l2,l3]
l0,l1,l2,l3=0,0,0,0
for i in prelabel[a+b+c:a+b+c+d]:
    if i==0:
        l0=l0+1
    if i==1:
        l1=l1+1
    if i==2:
        l2=l2+1
    if i==3:
        l3=l3+1  
L3=[l0,l1,l2,l3]
c = np.array([L0,L1,L2,L3])
print c
print (342.0/c[0].sum(),308.0/c[1].sum(),9.0/c[2].sum(),212.0/c[3].sum())

[[ 89  67 342 244]
 [ 21 308  31 117]
 [  9   5   1  19]
 [205 278  23 212]]
(0.4609164420485175, 0.6457023060796646, 0.2647058823529412, 0.29526462395543174)


In [9]:
labels = label
pca = PCA(n_components=5).fit(result.reshape(result.shape[0],-1))
bench_k_means(KMeans(init=pca.components_, n_clusters=5, n_init=1),
              name="PCA-based",
              data=result.reshape(result.shape[0],-1))

PCA-based   0.74s    10453291529   0.271   0.196   0.227   0.198   0.194    0.077


In [13]:
c = np.array([[1, 2, 3, 4],[4, 5, 6, 7], [7, 8, 9, 10]])
d=c.reshape(3,-1)
print d

[[ 1  2  3  4]
 [ 4  5  6  7]
 [ 7  8  9 10]]


In [11]:
data = result.reshape(result.shape[0],-1)
n_samples, n_features = data.shape
print n_samples, n_features

1971 3072


In [23]:
a.labels_

array([1, 2, 2, ..., 2, 3, 2], dtype=int32)